## Erode the surface during He-3 production/diffusion
<br>
  
#### Syntax
`conc3He = HeErodeFxn(conc0, erode_z, CosmoPars, HeliumPars)` <br>
#### Input 
`conc0` : concentration at the beginning of the "year" (at g-1) <br>
`erode_z` : depth to be eroded (cm) <br>
`CosmoPars` : dictionary of parameters relevant for CRN calculations <br>
`HeliumPars` : dictionary of parameters relevant for He-3 calculations <br>
#### Variables Used
##### Helium Pars
`Hez` : vector of He node depths, size length(max_depths[0]) (cm) <br>
`EDTz` : vector of EDTs at the Hez node depths (K) <br>
`nx` : number of nodes in the quartz grains <br>
##### Cosmo Pars    
`mu` : mu production term <br>
`SLHL_He3` : sea level high latitude He-3 spallation production rate (at g-1 yr-1) <br>
#### Output
##### Calc Pars
`conc3He` : matrix size Hez, nx with concentration of 3He at depth and in each quartz grain <br>
#### Notes
**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [4]:
def HeErodeFxn(conc0, erode_z, CosmoPars, HeliumPars):
    Hez = HeliumPars['Hez']
    EDTz = HeliumPars['EDTz']
    nx = HeliumPars['nx']
    
    mu = CosmoPars['mu']
    SLHL_He3 = CosmoPars['SLHL_He3']
    
    old_depths = Hez[Hez > erode_z]
    He3P0 = SLHL_He3 * np.exp(-mu * Hez)

    conc0 = [ProdDiffHe3Fxn(conc0[i], He3P0[i], EDTz[i], HeliumPars) for i in range(len(Hez))] 
    
    interpx = range(0,nx)
    first_fxn = interp2d(x = interpx, 
                         y = Hez, 
                         z = np.stack(conc0), 
                         kind = 'linear')    
    
    first_data = first_fxn(x = interpx, y = Hez)
    
    second_fxn = interp2d(x = interpx, 
                          y = (Hez - erode_z), 
                          z = np.stack(first_data), 
                          kind = 'linear') 
    
    conc3He = second_fxn(x = interpx, y = Hez)
    
    return conc3He